In [ ]:
import sys
print(sys.executable)

C:\Users\hp\AppData\Local\Programs\Python\Python310\python.exe


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 103.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

In [1]:
import os
# Data processing
import pandas as pd
import numpy as np
# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
# Topic model
from bertopic import BERTopic
# Dimension reduction
from umap import UMAP
import gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
os.chdir("./theory")

In [3]:
os.chdir("./DBMS")

In [6]:
content = pd.read_csv('train.csv', sep='\t', names=['review'], encoding='cp1252')

content.head()

,review
0,Databases and Database Users Databases and dat...
1,For example if we go to the bank to deposit o...
2,Even purchasing items at a supermarket often ...
3,These interactions are examples of what we ma...
4,In the past few years advances in technology ...


In [7]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13851 entries, 0 to 13850
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  13851 non-null  object
dtypes: object(1)
memory usage: 108.3+ KB


In [8]:
# content['review'] = content['review'].apply(lambda x: ' '.join([w for w in gensim.utils.simple_preprocess(x) if w.lower() not in gensim.parsing.preprocessing.STOPWORDS and len(w) > 3]))
# Lemmatization
content['review'] = content['review'].apply(lambda x: ' '.join([wn.lemmatize(w) for w in gensim.utils.simple_preprocess(x) if w not in gensim.parsing.preprocessing.STOPWORDS and len(w) > 2]))

In [9]:
content.head()


,review
0,database database user database database syste...
1,example bank deposit withdraw fund hotel airli...
2,purchasing item supermarket automatically upda...
3,interaction example traditional database appli...
4,past year advance technology led exciting new ...


In [10]:
content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13851 entries, 0 to 13850
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  13851 non-null  object
dtypes: object(1)
memory usage: 108.3+ KB


In [ ]:
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)
# Run BERTopic model
topics, probabilities = topic_model.fit_transform(content['review'])

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
import numpy as np
import torch

# Prepare embeddings
docs = content['review']
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)

# # Train our topic model using our pre-trained sentence-transformers embeddings
# umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
# topic_model = BERTopic(umap_model=umap_model, embedding_model=sentence_model)
# topics, probs = topic_model.fit_transform(docs, embeddings)

# Generate `X` and `labels` only for non-outlier topics (as they are technically not clusters)
umap_embeddings = no_len_topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

# Calculate silhouette score
silhouette_score(X, labels)

In [ ]:
import gensim.corpora as corpora
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
documents = pd.DataFrame({"Document": content['review'],
                          "ID": range(len(filtered_text)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='u_mass')
coherence = coherence_model.get_coherence()

In [11]:
topic_model.get_topic_info()


,Topic,Count,Name
0,-1,8609,-1_time_packet_network_message
1,0,576,0_telephone_phone_voice_mobile
2,1,309,1_fiber_cable_optical_light
3,2,234,2_byte_payload_length_field
4,3,231,3_agent_mail_email_smtp
...,...,...,...
329,328,10,328_title_font_amalgamated_head
330,329,10,329_file_uploader_archive_edit
331,330,10,330_appended_remainder_quotient_delineation
332,331,10,331_tag_attribute_closing_annotation


In [14]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('telephone', 0.03493154269159345),
 ('phone', 0.03157699192742084),
 ('voice', 0.031160691862054033),
 ('mobile', 0.030184707696185364),
 ('call', 0.01535548431196776),
 ('office', 0.014191768056895272),
 ('toll', 0.012306492754446246),
 ('mobility', 0.011702681020533914),
 ('company', 0.011530526675275349),
 ('trunk', 0.008861242120914837)]

In [40]:
chart_bar = topic_model.visualize_barchart(top_n_topics=30)
chart_bar.write_html('theory/Networking/Graphs/chart_bar.html')

topic_model.visualize_barchart(top_n_topics=30)

In [41]:
# Visualize intertopic distance
chart_topic_similar = topic_model.visualize_topics()
chart_topic_similar.write_html('theory/Networking/Graphs/chart_top_similar.html')

topic_model.visualize_topics()

In [43]:
# Visualize connections between topics using hierachical clustering
chart_hier = topic_model.visualize_hierarchy(top_n_topics=50)
chart_hier.write_html('theory/Networking/Graphs/chart_hier.html')

topic_model.visualize_hierarchy(top_n_topics=50)

In [44]:
chart_heat = topic_model.visualize_heatmap()
chart_heat.write_html('theory/Networking/Graphs/chart_heat.html')

topic_model.visualize_heatmap()

Output hidden; open in https://colab.research.google.com to view.

In [45]:
# Visualize probability distribution
chart_dist = topic_model.visualize_distribution(topic_model.probabilities_[0], min_probability=0.0001)
chart_dist.write_html('theory/Networking/Graphs/chart_dist.html')

topic_model.visualize_distribution(topic_model.probabilities_[0], min_probability=0.0001)

In [23]:
content['review'][0]


'century great mechanical system accompanying industrial revolution'

In [24]:
# Get probabilities for all topics
topic_model.probabilities_[0]

array([6.90377869e-04, 4.31870454e-04, 5.60477725e-04, 5.73492173e-04,
       4.38832976e-04, 6.43409785e-04, 6.92864168e-04, 4.72984315e-04,
       4.99054747e-04, 5.99081779e-04, 4.12162552e-04, 3.99409720e-04,
       4.74157841e-04, 4.44118857e-04, 4.51150173e-04, 4.36172534e-04,
       8.85614503e-05, 5.33718475e-04, 5.76285356e-04, 5.15906542e-04,
       1.74266474e-03, 3.67065346e-04, 3.56863137e-04, 5.01468131e-04,
       5.40532808e-04, 3.65739953e-04, 3.88156440e-04, 3.99184189e-04,
       4.73290360e-04, 5.54698747e-04, 3.96625892e-04, 5.72198366e-04,
       3.94749061e-04, 5.68226351e-04, 3.48786449e-04, 5.79164785e-04,
       3.84599627e-04, 6.49889766e-04, 1.71386905e-04, 6.54230148e-04,
       4.43607846e-04, 4.83174240e-04, 1.22133001e-03, 3.40245851e-04,
       1.67077877e-04, 4.25570837e-04, 7.44514221e-04, 4.58811897e-04,
       5.37445481e-04, 4.58842177e-04, 3.53365372e-04, 4.71535045e-04,
       4.10691880e-04, 4.80594913e-04, 3.03069371e-04, 4.50400562e-04,
      

In [28]:
#this saves the predicted topics for each row in the csv file
topic_prediction = topic_model.topics_[:]
print(topic_prediction)

[-1, -1, -1, -1, -1, -1, -1, 153, -1, -1, -1, -1, -1, -1, 64, 209, -1, 39, 299, 151, -1, -1, -1, 151, -1, 151, 172, 44, -1, -1, 299, 148, -1, -1, -1, 157, 151, -1, 151, -1, -1, -1, -1, 39, 0, -1, -1, -1, -1, -1, -1, -1, 261, -1, -1, -1, 0, -1, -1, 221, -1, -1, -1, -1, -1, -1, 135, -1, 46, -1, -1, 39, 160, -1, 46, 46, -1, -1, 46, -1, -1, 46, 46, -1, -1, 105, 123, -1, -1, -1, -1, 3, 123, 0, 0, 0, 0, -1, -1, -1, -1, 155, -1, 155, 305, 189, -1, 145, -1, 221, 221, 221, -1, -1, 91, -1, -1, -1, -1, 64, -1, -1, -1, -1, -1, -1, 98, -1, -1, -1, 106, -1, -1, -1, 98, -1, 170, 198, 170, 171, -1, -1, 46, -1, 35, -1, -1, -1, -1, 189, 197, 141, -1, 197, -1, 201, -1, 0, -1, -1, -1, 0, -1, -1, 191, 191, -1, -1, -1, -1, 98, -1, 98, -1, -1, 155, 98, -1, 221, 221, -1, -1, -1, 82, -1, 221, -1, -1, 331, -1, 221, -1, -1, 191, 197, 1, 191, 18, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 180, 254, -1, 18, 18, 1, -1, -1, 13, 13, 13, 13, 13, 13, 13, 64, 0, -1, 0, -1, -1, 0, 18, 0, 18, 0, -1, 18, -1, 135, -1, -1, 

In [35]:
#for predicting the topics for any new sentence
new_check = 'What do you mean by network topology'

# Find topics
num_of_topics = 3
similar_topics, similarity = topic_model.find_topics(new_check, top_n=num_of_topics)

print(f'The top {num_of_topics} similar topics are {similar_topics}, and the similarities are {np.round(similarity,2)}')

#for printing those 3 topics
for index, top in enumerate(similar_topics):
  keys = [t[0] for t in topic_model.get_topic(top)]
  print(f'{keys} : with probability \n{np.round(similarity,2)[index]}')



The top 3 similar topics are [39, 151, 26], and the similarities are [0.49 0.46 0.42]
['networking', 'network', 'autonomous', 'consider', 'regional', 'interconnected', 'dessert', 'villain', 'freenet', 'spinach'] : with probability 
0.49
['interconnected', 'internetworks', 'collection', 'internetwork', 'internet', 'autonomous', 'distributed', 'straight', 'internetworking', 'distinction'] : with probability 
0.46
['layer', 'network', 'deal', 'interface', 'isolating', 'linchpin', 'overlaid', 'internet', 'imperfection', 'overcome'] : with probability 
0.42


In [36]:
# Save the topic model
topic_model.save("/content/drive/MyDrive/placement/theory/Networking/networking_trained_model")	

# Load the topic model
# my_model = BERTopic.load("amz_review_topic_model")

/usr/local/lib/python3.9/dist-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [37]:
import pickle

with open('/content/drive/MyDrive/placement/theory/Networking/networking_trained_model.pkl', 'wb') as f:
    pickle.dump(topic_model, f)


In [38]:
#for using saved pickle model
# import pickle

# with open('bert_model.pkl', 'rb') as f:
#     bert_model = pickle.load(f)
